# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Mini-Experiments

In [0]:
Y.head()

,attack_cat
0,Exploits
1,Exploits
2,Reconnaissance
3,Exploits
4,Exploits


In [0]:
len(Y.attack_cat.unique())

9

In [0]:
X[X.columns[1]].values

array([-0.45662979,  1.81115277,  1.81115277, ...,  4.07893533,
        4.07893533,  4.07893533])

In [0]:
X.iloc[:, [1]].values

array([[-0.45662979],
       [ 1.81115277],
       [ 1.81115277],
       ...,
       [ 4.07893533],
       [ 4.07893533],
       [ 4.07893533]])

In [0]:
print(X.shape[1])
print(len(X.columns))
print(X.columns[1])

43
43
state


In [0]:
Y.iloc[:, [0]].values

array([['Exploits'],
       ['Exploits'],
       ['Reconnaissance'],
       ...,
       ['DoS'],
       ['DoS'],
       ['Exploits']], dtype=object)

# Feature Engineering

In [0]:
%%capture warnings1
# cluster and score
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import homogeneity_score as hs

score = []
for i in range(len(X.columns)): # loop number of features
  K = KMeans(n_clusters=len(Y.attack_cat.unique())-1, random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = hs(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [0]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , Homogeneity Score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , Homogeneity Score

(9, 0.5200499083860681)
(41, 0.4128816872291432)
(37, 0.4078111262210801)
(36, 0.3994266255429144)
(40, 0.39565162191059955)
(42, 0.3815622954439636)
(38, 0.371542709916172)
(39, 0.31191652718297214)
(18, 0.2879622288588891)
(10, 0.23990215273135584)
(19, 0.22462936757191546)
(32, 0.20008929682355084)
(1, 0.1998217658223299)
(28, 0.19497974899658832)
(6, 0.19401906633262922)
(14, 0.18929063032108048)
(15, 0.18927970359288043)
(30, 0.18760670022656042)
(17, 0.17415486183187892)
(16, 0.17304877589910456)
(29, 0.16959218396034223)
(2, 0.14903041194944955)
(0, 0.12101295319877158)
(27, 0.10929660250895264)
(5, 0.0986455027153642)
(22, 0.0909767556288571)
(33, 0.04843865688159797)
(20, 0.046156044993459586)
(23, 0.035925992349364125)
(11, 0.02877177059375085)
(24, 0.027800651687912976)
(25, 0.027792728735092986)
(26, 0.011667271015717523)
(34, 0.009117026092838924)
(35, 0.009117026092838924)
(12, 0.007689908300971081)
(8, 0.006896830390760858)
(4, 0.006

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [0]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6500    0.1680    0.2670       619
      Backdoor     0.6000    0.0741    0.1319       567
           DoS     0.3166    0.2724    0.2928      4075
      Exploits     0.6337    0.8070    0.7099     11236
       Fuzzers     0.8984    0.8632    0.8805      6127
       Generic     0.9968    0.9860    0.9913     53886
Reconnaissance     0.9194    0.7448    0.8229      3385
     Shellcode     0.6707    0.5773    0.6205       388
         Worms     1.0000    0.2895    0.4490        38

      accuracy                         0.8902     80321
     macro avg     0.7428    0.5314    0.5740     80321
  weighted avg     0.8937    0.8902    0.8873     80321



In [0]:
%%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

In [0]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.8919183902142958 max accuracy
0 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [0]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
x = list(zip(X.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X.columns)):
    print(x[i])

('state', 0.15884655607217285)
('ct_src_ ltm', 0.13744959916446162)
('proto', 0.11870576441654455)
('dur', 0.07865193158153869)
('Sjit', 0.06792370083470976)
('dbytes', 0.052347305301381014)
('dloss', 0.05080842387179985)
('sloss', 0.04280245301928314)
('Spkts', 0.02882349492666332)
('Stime', 0.025092295473842344)
('Sload', 0.02002885012745306)
('service', 0.016492258854834182)
('sttl', 0.014398228264773948)
('sbytes', 0.013737540817738446)
('ct_srv_dst', 0.012677144008360732)
('is_sm_ips_ports', 0.01201462264827744)
('ackdat', 0.011625093793147794)
('ct_dst_ltm', 0.01129591322101133)
('trans_depth', 0.01124523779668543)
('Ltime', 0.01102683169183212)
('Djit', 0.010176763558516495)
('ct_state_ttl', 0.009828293072511032)
('ct_srv_src', 0.009104784470492969)
('res_bdy_len', 0.009078789363032093)
('cprtt', 0.008013367982067835)
('synack', 0.007690303631676143)
('dtcpb', 0.0070275413357082355)
('smeansz', 0.006814649851549499)
('ct_ftp_cmd', 0.00671756127919985)
('ct_src_dport_ltm', 0.0064

In [0]:
clf.score(x_train, y_train) 

0.9284391462342331

In [0]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.8919183902142958

In [0]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.7459    0.1838    0.2950       495
      Backdoor     0.5584    0.0917    0.1575       469
           DoS     0.3186    0.2674    0.2908      3235
      Exploits     0.6358    0.8112    0.7129      8944
       Fuzzers     0.8934    0.8702    0.8817      4846
       Generic     0.9970    0.9864    0.9916     43219
Reconnaissance     0.9324    0.7422    0.8265      2696
     Shellcode     0.6746    0.6199    0.6461       321
         Worms     0.9091    0.3125    0.4651        32

      accuracy                         0.8919     64257
     macro avg     0.7406    0.5428    0.5852     64257
  weighted avg     0.8952    0.8919    0.8889     64257

